In [10]:
import pandas as pd
import numpy as np
import pygeohash as pgh

import seaborn as sns
from gensim.models import Word2Vec
from sklearn.metrics import silhouette_score
from sklearn.cluster import OPTICS
import hdbscan

from tqdm.notebook import tqdm

In [3]:
df = pd.read_csv('../data/worked_data.csv')

In [4]:
df.head()

,Unnamed: 0,product_id,name,unit_price,geohash,primary_cuisine,en_name
0,1,ec33db14a2c5,alcachofas empinzadas,0.304,u6scd,spanish,alcachofas empinzadas
1,2,fa04d9e5b964,coca-cola 2l,0.240,u6scq,pizza,coca-cola 2l
2,3,9f502de9373e,"5 lax, 5 räkor, 5 avokado",0.820,u6sck,sushi,"5 salmon, 5 shrimp, 5 avocados"
3,5,35ac685bcc43,52 femtiotvå,0.872,u6sct,thai,52 fifty-two
4,6,ea2ce1a7db0f,sprite 33 cl,0.152,u6sct,thai,sprite 33 cl


In [5]:
lat = []
long = []
for index, row in df.iterrows(): 
    x, y = pgh.decode(row['geohash'])
    lat.append(x)
    long.append(y)
df['latitude'] = lat
df['longitude'] = long

In [6]:
df = df.drop(columns = ['Unnamed: 0', 'name', 'geohash'])

In [7]:
df.head()

,product_id,unit_price,primary_cuisine,en_name,latitude,longitude
0,ec33db14a2c5,0.304,spanish,alcachofas empinzadas,59.3,18.0
1,fa04d9e5b964,0.240,pizza,coca-cola 2l,59.3,18.2
2,9f502de9373e,0.820,sushi,"5 salmon, 5 shrimp, 5 avocados",59.3,18.1
3,35ac685bcc43,0.872,thai,52 fifty-two,59.3,18.2
4,ea2ce1a7db0f,0.152,thai,sprite 33 cl,59.3,18.2


In [8]:
df.isna().value_counts()

product_id  unit_price  primary_cuisine  en_name  latitude  longitude
False       False       False            False    False     False        96575
                                         True     False     False            3
dtype: int64

In [9]:
df = df.fillna('')
data = [df['primary_cuisine'].to_list() + df['en_name'].to_list()]

## Выбор оптимальной длины вектора эмбединга

In [13]:
sizes = [7, 10, 15, 20]
best_size = 0
best_metric = 0

for size in tqdm(sizes):
    #эмбединг
    w2v_model = Word2Vec(min_count=1, window=1, vector_size = size)
    w2v_model.build_vocab(data)
    w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
    new_df = df.copy()
    new_df = new_df.drop(columns = ['primary_cuisine', 'en_name', 'product_id'])

    for index, row in df.iterrows(): 
        primary_cuisine = w2v_model.wv[row['primary_cuisine']]
        en_name = w2v_model.wv[str(row['en_name'])]
        i = 0
        for n in primary_cuisine:
            new_df.at[index, f'primary_cuisine_{i}'] = n
            i += 1
        i = 0
        for n in en_name:
            new_df.at[index, f'en_name_{i}'] = n
            i += 1
    #hdbscan
    clusterer = hdbscan.HDBSCAN(min_cluster_size=20, gen_min_span_tree=True)
    clusterer.fit(new_df)
    score = silhouette_score(new_df, clusterer.labels_)
    if score > best_metric:
        best_metric = score
        best_size = size
    print('hdbscan', score)

  0%|          | 0/4 [00:00<?, ?it/s]

hdbscan 0.0928351040251084
hdbscan 0.1364274458589733
hdbscan 0.13598341868929356
hdbscan 0.12024419003604549


In [14]:
best_size

10

## Выбор минимального размера кластера

In [16]:
cluster_sizes = [20, 50, 100, 150, 200, 250]
cluster_best_size = 0
best_metric = 0

#эмбединг
w2v_model = Word2Vec(min_count=1, window=1, vector_size = 10)
w2v_model.build_vocab(data)
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
new_df = df.copy()
new_df = new_df.drop(columns = ['primary_cuisine', 'en_name', 'product_id'])

for index, row in df.iterrows(): 
    primary_cuisine = w2v_model.wv[row['primary_cuisine']]
    en_name = w2v_model.wv[str(row['en_name'])]
    i = 0
    for n in primary_cuisine:
        new_df.at[index, f'primary_cuisine_{i}'] = n
        i += 1
    i = 0
    for n in en_name:
        new_df.at[index, f'en_name_{i}'] = n
        i += 1

for size in tqdm(cluster_sizes):
    #hdbscan
    clusterer = hdbscan.HDBSCAN(min_cluster_size=size, gen_min_span_tree=True)
    clusterer.fit(new_df)
    score = silhouette_score(new_df, clusterer.labels_)
    if score > best_metric:
        best_metric = score
        best_size = size
    print(f'cluster_size={size} score={score}')

  0%|          | 0/6 [00:00<?, ?it/s]

cluster_size=20 score=0.1364274458589733
cluster_size=50 score=0.5162352593415073
cluster_size=100 score=0.7087550841230811
cluster_size=150 score=0.7094847195208035
cluster_size=200 score=0.7041127757159253
cluster_size=250 score=0.7041210706354343


### Коэффициент Силуэтта 

К-т Силуэтта (англ *Silhouette*) - это метрика, которая не предполагает знания истинных меток объектов, и позволяет оценить качество кластеризации, используя только саму (неразмеченную) выборку и результат кластеризации.

Интуитивное описание метрики:
* точки внутри кластера должны лежать очень близко друг к другу, то есть кластер должен быть *плотным*
* сами кластера должны лежать как можно дальше друг от друга

Метрика силуэта позволяет учитывать оба этих факта в одной формуле.

Чтобы вычислить его для каждого объекта нужно вычислить для каждого объекта выборки две величины, $a$ (среднее расстояние от данного объекта до объектов из того же кластера) и $b$ (среднее расстояние от данного объекта до объектов из ближайшего кластера (отличного от того, в котором лежит сам объект)). 

Силуэтом объекта назовём нормализованную разность между этими величинами

$$
s = \frac{b-a}{\max(a,b)}
$$

Силуэтом выборки называется средняя величина силуэта объектов данной выборки. Таким образом, силуэт показывает, насколько среднее расстояние до объектов своего кластера отличается от среднего расстояния до объектов других кластеров. Эта величина меняется в интервале от $-1$ до $1$:

* -1 значит что кластера плохие, размытые
* 0 значит что кластера накладываются друг на друга
* 1 значит что кластера плотные и хорошо отделены друг от друга

Таким образом, чем ближе значение к-та Силуэтта в единице, тем лучше. Все, что больше $0.5$ хорошие значения, всё что меньше - надо улучшать.

Формулу реализовывать не надо, она уже есть в *sklearn*. Для примера загрузим наш датасет с кластеризацией:

## Обучение алгоритма и создание колонки кластера в данных

In [17]:
w2v_model = Word2Vec(min_count=1, window=1, vector_size = 10)
w2v_model.build_vocab(data)
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
new_df = df.copy()
new_df = new_df.drop(columns = ['primary_cuisine', 'en_name', 'product_id'])

for index, row in df.iterrows(): 
    primary_cuisine = w2v_model.wv[row['primary_cuisine']]
    en_name = w2v_model.wv[str(row['en_name'])]
    i = 0
    for n in primary_cuisine:
        new_df.at[index, f'primary_cuisine_{i}'] = n
        i += 1
    i = 0
    for n in en_name:
        new_df.at[index, f'en_name_{i}'] = n
        i += 1

clusterer = hdbscan.HDBSCAN(min_cluster_size=100, gen_min_span_tree=True)
clusterer.fit(new_df)

HDBSCAN(gen_min_span_tree=True, min_cluster_size=100)

In [21]:
new_df['cluster'] = clusterer.labels_
new_df['product_id'] = df['product_id']

In [22]:
new_df.to_csv('../data/with_clusters.csv')

In [23]:
new_df.head()

,unit_price,latitude,longitude,primary_cuisine_0,primary_cuisine_1,primary_cuisine_2,primary_cuisine_3,primary_cuisine_4,primary_cuisine_5,primary_cuisine_6,...,en_name_2,en_name_3,en_name_4,en_name_5,en_name_6,en_name_7,en_name_8,en_name_9,cluster,product_id
0,0.304,59.3,18.0,0.706078,-1.448574,1.675925,-1.972668,-0.204048,0.559646,4.476071,...,0.083150,-0.042129,0.031928,0.026733,-0.029407,-0.074990,0.098144,-0.031006,5,ec33db14a2c5
1,0.240,59.3,18.2,0.388050,-1.036243,2.172207,-1.290226,0.246125,-0.469956,7.482125,...,0.029254,-0.090641,0.085059,0.072139,0.078608,0.096100,-0.086711,-0.021956,20,fa04d9e5b964
2,0.820,59.3,18.1,-0.160905,-1.277902,2.375871,-2.297812,0.321750,-0.944451,6.697765,...,0.009403,-0.049864,0.057560,-0.011735,-0.033440,-0.077631,-0.072010,-0.001155,31,9f502de9373e
3,0.872,59.3,18.2,0.427666,-1.088354,2.023588,-1.891981,0.932764,-0.032216,6.375370,...,0.010731,-0.089043,-0.086058,-0.055701,-0.090528,-0.006858,-0.097648,-0.093000,26,35ac685bcc43
4,0.152,59.3,18.2,0.427666,-1.088354,2.023588,-1.891981,0.932764,-0.032216,6.375370,...,-0.072503,-0.096033,-0.027436,-0.083628,-0.060389,-0.056709,-0.023441,-0.017070,26,ea2ce1a7db0f
